empezamos a trabajar con gradientes  con la función R^3 -> R, z=f(x)


In [1]:
# gradientes
import torch
x = torch.tensor([1.0, 2, 3], requires_grad=True)  

exponent = torch.tensor([1, 2, 3])     # tensor con potencias

z1 = torch.pow(x * 2, exponent)        # eleva cada elemento de (x*2) a la potencia correspondiente

print(z1)   # si tenemos algo distinto a un escalar, no podra calcular el gradiente
print(f"z1 {z1}  y su shape es {z1.shape}")     # esto es un vector, no un escalar
z = z1.mean()   # esto es un escalar
print(z)

# Primera derivada dz/dx, creando grafo para derivadas superiores
z.backward(create_graph=True)  
print(f"Primera derivada x.grad: {x.grad}")

#otra forma de referirse a la derivada
grad = torch.autograd.grad(z, x, create_graph=True)[0]

# Problem: calling .backward() twice on the same computation graph without clearing the gradients. 
# By default, PyTorch accumulates gradients in the .grad attribute of tensors when .backward() is called. 
# This means that calling .backward() multiple times without zeroing out the gradients can result in incorrect values because the gradients from 
# the previous call are added to the new gradients.

hessian = torch.zeros(x.size(0), x.size(0))  # matriz Hessiana inicializada en ceros

for i in range(x.size(0)):
    # Calculamos gradiente de grad[i] respecto a x (segunda derivada)
    grad2 = torch.autograd.grad(grad[i], x, retain_graph=True)[0]
    hessian[i] = grad2

print("Hessiana completa:")
print(hessian)


# Cálculos con la segunda derivada: gradiente de x.grad respecto a x
cal_segunda = torch.autograd.grad(outputs=x.grad, inputs=x, grad_outputs=torch.ones_like(x), retain_graph=True)[0]
print(f"cálculos con la segunda derivada: {cal_segunda}")

# En PyTorch, cuando tienes dos funciones f(x) y g(x) que dependen del mismo tensor x
# con requires_grad=True, y calculas los gradientes de ambas, los gradientes se acumulan 
# en el atributo x.grad a menos que los limpies explícitamente


tensor([  2.,  16., 216.], grad_fn=<PowBackward1>)
z1 tensor([  2.,  16., 216.], grad_fn=<PowBackward1>)  y su shape es torch.Size([3])
tensor(78., grad_fn=<MeanBackward0>)
Primera derivada x.grad: tensor([ 0.6667,  5.3333, 72.0000], grad_fn=<CopyBackwards>)
Hessiana completa:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  2.6667,  0.0000],
        [ 0.0000,  0.0000, 48.0000]])


c:\Users\econo\.conda\envs\cursopython2\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\autograd\engine.cpp:1206.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


cálculos con la segunda derivada: tensor([ 0.0000,  2.6667, 48.0000])
